In [1]:
import csv
import numpy as np
from copy import deepcopy
from math import gamma

In [2]:
# Constitution des bases de données
L_motorisation=["Electrique","Hybride rechargeable","Hydrogène","GNV","Autres fossiles"]
d_motorisation={"Electrique":0,"Hybride rechargeable":1,"Hydrogène":2,"GNV":3,"Autres fossiles":4}
L_vp_ages=[2.01,2.97,0,10.42,10.4]
L_vul_ages=[3.42,1.22,0,11,9.98]
L_bus_ages=[5.06,14.04,0,7.31,8.39]
L_car_ages=[2.74,0,0,1.39,7.63]
L_pl_ages=[5.1,0,0,2.5,9.1]
L_vp_parc_2021=[244863,
157446,
0,
153769,    
37790188]
L_vul_parc_2021=[48659,
747,
0,
15954,
5839037]
L_bus_parc_2021=[869,72,0,3496,23022]
L_car_parc_2021=[69,0,0,584,65393]
L_pl_parc_2021=[139,0,0,6603,593541]
duree_vie_vp=20
duree_vie_vul=20
duree_vie_bus=15
duree_vie_car=15
duree_vie_pl=18
k_vp=2
k_vul=2
k_bus=2
k_car=2
k_pl=2

year_ref=2021

n_motor=len(L_motorisation)

def age(alpha,d):
    s=sum([1/gamma(1+alpha*(i+1)) for i in range(d)])
    return sum([(i+1/2)*1/gamma(1+alpha*(i+1)) for i in range(d)])/s
    
def find_alpha(age_moy,duree_vie):
    d=2*duree_vie
    alpha0=0
    alpha1=1
    eps=1e-3
    while age(alpha1,d)>age_moy:
        alpha1+=1
    while age(alpha0,d)>age_moy+eps:
        alpha=(alpha0+alpha1)/2
        if age(alpha,d)>age_moy:
            alpha0=alpha
        else:
            alpha1=alpha
    return alpha0

def make_database(L_ages,L_parc,duree_vie):
    L_base=[]
    for i in range(n_motor):
        if L_parc[i]>0:
            d=2*duree_vie
            alpha=find_alpha(L_ages[i],duree_vie)
            L_distrib=[1/gamma(1+alpha*(i+1)) for i in range(d)]
            s=sum(L_distrib)
            for year in range(year_ref-d,year_ref):
                L_base.append([L_motorisation[i],year,round(L_distrib[year_ref-year-1]/s*L_parc[i])])
                    #b*q**(year_ref-year-1)*np.exp(-((year_ref-year-1)/lamb)**k))])
    return L_base

L_base_vp=make_database(L_vp_ages,L_vp_parc_2021,duree_vie_vp)
L_base_vul=make_database(L_vul_ages,L_vul_parc_2021,duree_vie_vul)
L_base_bus=make_database(L_bus_ages,L_bus_parc_2021,duree_vie_bus)
L_base_car=make_database(L_car_ages,L_car_parc_2021,duree_vie_car)
L_base_pl=make_database(L_pl_ages,L_pl_parc_2021,duree_vie_pl)

In [16]:
## Evolution du parc

#L_vp_new_2021=[0.0972,0.0827,0,0.0272,0.7929]
#L_vul_new_2021=[0.0252,0.0061,0,0.0029,0.9658]
#L_vp_new_2030=[0.5,0.1,0.0015,0,0.3985]
#L_vul_new_2030=[0.49,0.01,0.005,0,0.495]
#L_vp_new_2035=[0.997,0,0.003,0,0]
#L_vul_new_2035=[0.99,0,0.01,0,0]
L_vp_new=[[0.0972,0.0827,0,0.0272,0.7929],
         [0.6,0.1,0.0015,0,0.2985],
         [0.997,0,0.003,0,0]]
L_vp_years_new=[2021,2030,2035]
L_vul_new=[[0.0252,0.0061,0,0.0029,0.9658],
          [0.49,0.01,0.005,0,0.495],
          [0.99,0,0.01,0,0]]
L_vul_years_new=[2021,2030,2035]


L_vp_new_fit55=[[0.0972,0.0827,0,0.0272,0.7929],
         [0.65,0.1,0.0015,0,0.2485],
         [0.997,0,0.003,0,0]]
L_vp_years_new_fit55=[2021,2025,2030]
L_vul_new_fit55=[[0.0252,0.0061,0,0.0029,0.9658],
          [0.5,0.01,0.005,0,0.485],
          [0.99,0,0.01,0,0]]
L_vul_years_new_fit55=[2021,2025,2030]

L_bus_new=[[0.2813,0,0.0037,0.4926,0.2224],
         [0.37,0,0.03,0.6,0],
         [0.5,0,0.1,0.4,0],
         [0.8,0,0.15,0.05,0]]
L_bus_years_new=[2021,2025,2030,2040]
L_car_new=[[0.0017,0,0,0.1422,0.8561],
         [0.03,0,0.02,0.35,0.6],
         [0.1,0,0.15,0.55,0.2],
         [0.3,0,0.4,0.3,0]]
L_car_years_new=[2021,2025,2030,2040]

L_pl_new=[[0.0015,0,0.0009,0.0371,0.9605],
         [0.0566,0,0.0334,0.3,0.61],
         [0.1384,0,0.0816,0.3,0.38],
         [0.413,0,0.2435,0.3435,0]]
L_pl_years_new=[2021,2025,2030,2040]
    
def taux_new(L_new,L_years_new,year):
    if year<=L_years_new[0]:
        return L_new[0]
    elif year>=L_years_new[-1]:
        return L_new[-1]
    else:
        i=0
        while i<len(L_years_new) and year>=L_years_new[i]:
            i+=1
        L_taux=[]
        for j in range(n_motor-1):
            L_taux.append(L_new[i-1][j]+(year-L_years_new[i-1])/(L_years_new[i]-L_years_new[i-1])*(L_new[i][j]-L_new[i-1][j]))
        s=sum(L_taux)
        L_taux.append(1-s)
        return L_taux

N_new_vp_2021=1682582
N_new_vul_2021=423938
N_new_bus_2021=2154
N_new_car_2021=4627
N_new_pl_2021=44110
N_new_vp=2300000# Optimisé pour une sortie rapide des fossiles
N_new_vul=420000# Sur la base du CCFA 
#Voir https://ccfa.fr/wp-content/uploads/2021/02/analyse-statistiques-2020.pdf, p. 42
N_new_bus=2500
N_new_car=5500
N_new_pl=45000
taux_grow_vp=0
taux_grow_vul=0.004
taux_grow_bus=0.01#Historique 1,26 %
taux_grow_car=0.0036#Historique 
taux_grow_pl=0.0036


year_obj=2025# Retour à la normale automobile
def N_new_calc(N_new_2021,N_new,year):
    return round(N_new_2021+min((year-year_ref)/(year_obj-year_ref),1)*(N_new-N_new_2021))

def calc_parc(L_base):
    L_return=[0 for i in range(n_motor)]
    for i in range(len(L_base)):
        L_return[d_motorisation[L_base[i][0]]]+=L_base[i][2]
    return L_return

def nbr_sorties_foss(L_base,duree_vie,k,year,beta):
    N_sorties=0
    lamb=duree_vie/gamma(1+1/k)
    for i in range(len(L_base)):
        if L_base[i][0]=="Autres fossiles":
            N_sorties+=L_base[i][2]*(1-np.exp(beta*(-((year+1-L_base[i][1])/lamb)**k+((year-L_base[i][1])/lamb)**k)))
    return N_sorties

def find_beta(L_base,duree_vie,k,year,N_target):
    # Ne s'utilise que si le nombre de véhicules est >N_target
    eps=1
    beta0=0
    beta1=1
    while nbr_sorties_foss(L_base,duree_vie,k,year,beta1)<N_target:
        beta1+=1
    while nbr_sorties_foss(L_base,duree_vie,k,year,beta0)<N_target-eps:
        beta=(beta0+beta1)/2
        if nbr_sorties_foss(L_base,duree_vie,k,year,beta)<N_target:
            beta0=beta
        else:
            beta1=beta
    return beta0
    
def evol(L_base,L_new,L_years_new,N_new_2021,N_new,duree_vie,k,taux_grow,year):
    L_base_return=deepcopy(L_base)
        
    N_new_year=N_new_calc(N_new_2021,N_new,year)
    
    L_parc=calc_parc(L_base)
    N_base=sum(L_parc)
    
    # Mise au rebut des anciens véhicules
    # Vieillissement motorisations alternatives
    N_sortie_altern=0
    
    lamb=duree_vie/gamma(1+1/k)
    for i in range(len(L_base_return)):
        if L_base_return[i][0]!="Autres fossiles":
            n_sortie=np.random.binomial(L_base_return[i][2],1-np.exp(-((year+1-L_base_return[i][1])/lamb)**k+((year-L_base_return[i][1])/lamb)**k))
            L_base_return[i][2]-=n_sortie
            N_sortie_altern+=n_sortie
    # Vieillissement fossiles
    N_sortie=N_sortie_altern
    if L_parc[-1]>N_new_year-N_sortie_altern-taux_grow*N_base:
        beta=find_beta(L_base,duree_vie,k,year,N_new_year-N_sortie_altern-taux_grow*N_base)
        for i in range(len(L_base_return)):
            if L_base_return[i][0]=="Autres fossiles":
                n_sortie=np.random.binomial(L_base_return[i][2],1-np.exp(beta*(-((year+1-L_base_return[i][1])/lamb)**k+((year-L_base_return[i][1])/lamb)**k)))
                L_base_return[i][2]-=n_sortie
                N_sortie+=n_sortie
    else:
        for i in range(len(L_base_return)):
            if L_base_return[i][0]=="Autres fossiles":
                n_sortie=np.random.binomial(L_base_return[i][2],1-np.exp(-((year+1-L_base_return[i][1])/lamb)**k+((year-L_base_return[i][1])/lamb)**k))
                L_base_return[i][2]-=n_sortie
                N_sortie+=n_sortie

    # Correction à la baisse N_new_year pour respecter le taux de croissance max
    if N_new_year>taux_grow*N_base+N_sortie:
        N_new_year=round(taux_grow*N_base+N_sortie)
        
    # Arrivée des nouveaux véhicules (de l'année précédente)
    L_taux=taux_new(L_new,L_years_new,year-1)# Les véhicules aujoutés sont ceux de l'année year-1
    for i in range(n_motor):
        L_base_return.append([L_motorisation[i],year-1,round(N_new_year*L_taux[i])])
    return L_base_return


        

In [10]:
# Test
L_base_vp_test=evol(L_base_vp,L_vp_new,L_vp_years_new,N_new_vp_2021,N_new_vp,duree_vie_vp,k_vp,taux_grow_vp,2022)
print(calc_parc(L_base_vp_test))
L_base_bus_test=evol(L_base_bus,L_bus_new,L_bus_years_new,N_new_bus_2021,N_new_bus,duree_vie_bus,k_bus,taux_grow_bus,2022)
print(calc_parc(L_base_bus_test))
L_base_pl_test=evol(L_base_pl,L_pl_new,L_pl_years_new,N_new_bus_2021,N_new_pl,duree_vie_pl,k_pl,taux_grow_pl,2022)
print(calc_parc(L_base_pl_test))

[419604, 306342, 0, 196487, 37420528]
[1467, 70, 8, 4364, 21796]
[154, 0, 11, 6939, 595337]


In [11]:
# Execution VP-VUL
L_base_vpi=deepcopy(L_base_vp)
L_base_vuli=deepcopy(L_base_vul)
print(2021)
L_parc_evol_vp=[calc_parc(L_base_vpi)]
L_parc_evol_vul=[calc_parc(L_base_vuli)]
print(sum(L_parc_evol_vp[-1]))
print(sum(L_parc_evol_vul[-1]))
print(L_parc_evol_vp[-1])
print(L_parc_evol_vul[-1])
for year in range(2022,2061):
    L_base_vpi=evol(L_base_vpi,L_vp_new,L_vp_years_new,N_new_vp_2021,N_new_vp,duree_vie_vp,k_vp,taux_grow_vp,year)
    L_base_vuli=evol(L_base_vuli,L_vul_new,L_vul_years_new,N_new_vul_2021,N_new_vul,duree_vie_vul,k_vul,taux_grow_vul,year)
    print(year)
    L_parc_evol_vp.append(calc_parc(L_base_vpi))
    L_parc_evol_vul.append(calc_parc(L_base_vuli))
    print(sum(L_parc_evol_vp[-1]))
    print(sum(L_parc_evol_vul[-1]))
    print(L_parc_evol_vp[-1])
    print(L_parc_evol_vul[-1])

filename="Motorisations_vp.csv"
with open(filename,"w",newline='',encoding="utf-8") as f:
    fwrite=csv.writer(f, delimiter=';',quotechar='|')
    fwrite.writerow(["Année"]+L_motorisation)
    for year in range(2021,2061):
        fwrite.writerow([year]+L_parc_evol_vp[year-2021])
filename="Motorisations_vul.csv"
with open(filename,"w",newline='',encoding="utf-8") as f:
    fwrite=csv.writer(f, delimiter=';',quotechar='|')
    fwrite.writerow(["Année"]+L_motorisation)
    for year in range(2021,2061):
        fwrite.writerow([year]+L_parc_evol_vul[year-2021])  

2021
38346266
5904399
[244862, 157445, 0, 153769, 37790190]
[48660, 747, 0, 15954, 5839038]
2022
38345602
5928018
[419618, 306322, 0, 196377, 37423285]
[58279, 3319, 0, 16408, 5850012]
2023
38345602
5951730
[717847, 469469, 332, 236506, 36921448]
[89389, 6036, 234, 16691, 5839380]
2024
38345169
5975537
[1155317, 647229, 1043, 273328, 36268252]
[141722, 8872, 701, 16833, 5807409]
2025
38345170
5998582
[1746464, 839367, 2177, 305818, 35451344]
[214752, 11852, 1393, 16811, 5753774]
2026
38345170
6021964
[2455598, 1031900, 3686, 330358, 34523628]
[308319, 14953, 2308, 16697, 5679687]
2027
38344481
6045446
[3280356, 1224078, 5568, 347222, 33487257]
[421952, 18180, 3443, 16386, 5585485]
2028
38344481
6069628
[4216297, 1415274, 7784, 355885, 32349241]
[554920, 21512, 4798, 15902, 5472496]
2029
38344482
6093906
[5259924, 1604591, 10343, 357170, 31112454]
[706758, 24890, 6352, 15354, 5340552]
2030
38344482
6118282
[6406648, 1791386, 13228, 350740, 29782480]
[876846, 28273, 8115, 14667, 5190381]

In [18]:
# Execution VP fit-for-55
L_base_vpi=deepcopy(L_base_vp)
L_base_vuli=deepcopy(L_base_vul)
print(2021)
L_parc_evol_vp=[calc_parc(L_base_vpi)]
L_parc_evol_vul=[calc_parc(L_base_vuli)]
print(sum(L_parc_evol_vp[-1]))
print(sum(L_parc_evol_vul[-1]))
print(L_parc_evol_vp[-1])
print(L_parc_evol_vul[-1])
for year in range(2022,2061):
    L_base_vpi=evol(L_base_vpi,L_vp_new_fit55,L_vp_years_new_fit55,N_new_vp_2021,N_new_vp,duree_vie_vp,k_vp,taux_grow_vp,year)
    L_base_vuli=evol(L_base_vuli,L_vul_new_fit55,L_vul_years_new_fit55,N_new_vul_2021,N_new_vul,duree_vie_vul,k_vul,taux_grow_vul,year)
    print(year)
    L_parc_evol_vp.append(calc_parc(L_base_vpi))
    L_parc_evol_vul.append(calc_parc(L_base_vuli))
    print(sum(L_parc_evol_vp[-1]))
    print(sum(L_parc_evol_vul[-1]))
    print(L_parc_evol_vp[-1])
    print(L_parc_evol_vul[-1])

filename="Motorisations_vp_fit55.csv"
with open(filename,"w",newline='',encoding="utf-8") as f:
    fwrite=csv.writer(f, delimiter=';',quotechar='|')
    fwrite.writerow(["Année"]+L_motorisation)
    for year in range(2021,2061):
        fwrite.writerow([year]+L_parc_evol_vp[year-2021])
filename="Motorisations_vul_fit55.csv"
with open(filename,"w",newline='',encoding="utf-8") as f:
    fwrite=csv.writer(f, delimiter=';',quotechar='|')
    fwrite.writerow(["Année"]+L_motorisation)
    for year in range(2021,2061):
        fwrite.writerow([year]+L_parc_evol_vul[year-2021])

2021
38346266
5904399
[244862, 157445, 0, 153769, 37790190]
[48660, 747, 0, 15954, 5839038]
2022
38346061
5927789
[419536, 306334, 0, 196539, 37423652]
[58264, 3323, 0, 16404, 5849798]
2023
38346060
5951428
[881702, 474692, 746, 229334, 36759586]
[117730, 6268, 527, 16535, 5810368]
2024
38345580
5974299
[1670651, 662910, 2350, 250133, 35759536]
[226219, 9577, 1575, 16371, 5720557]
2025
38345579
5998196
[2822840, 871285, 4907, 256788, 34389759]
[382605, 13230, 3136, 15816, 5583409]
2026
38345579
6021478
[4276723, 1085439, 8290, 247148, 32727979]
[586755, 17215, 5214, 15021, 5397273]
2027
38345580
6045350
[5863297, 1248418, 12341, 237273, 30984251]
[828677, 20287, 7664, 14152, 5174570]
2028
38345051
6069532
[7580224, 1360089, 16978, 226919, 29160841]
[1106897, 22441, 10488, 13328, 4916378]
2029
38343502
6092786
[9418446, 1420438, 22248, 216222, 27266148]
[1420686, 23619, 13637, 12504, 4622340]
2030
38342067
6117157
[11372565, 1429601, 28068, 205739, 25306094]
[1768748, 23903, 17172, 1171

In [11]:
# Execution bus-cars
L_base_busi=deepcopy(L_base_bus)
L_base_cari=deepcopy(L_base_car)
print(2021)
L_parc_evol_bus=[calc_parc(L_base_busi)]
L_parc_evol_car=[calc_parc(L_base_cari)]
print(sum(L_parc_evol_bus[-1]))
print(sum(L_parc_evol_car[-1]))
print([x/sum(L_parc_evol_bus[-1]) for x in L_parc_evol_bus[-1]])
print([x/sum(L_parc_evol_car[-1]) for x in L_parc_evol_car[-1]])
for year in range(2022,2061):
    L_base_busi=evol(L_base_busi,L_bus_new,L_bus_years_new,N_new_bus_2021,N_new_bus,duree_vie_bus,k_bus,taux_grow_bus,year)
    L_base_cari=evol(L_base_cari,L_car_new,L_car_years_new,N_new_car_2021,N_new_car,duree_vie_car,k_car,taux_grow_car,year)
    print(year)
    L_parc_evol_bus.append(calc_parc(L_base_busi))
    L_parc_evol_car.append(calc_parc(L_base_cari))
    print(sum(L_parc_evol_bus[-1]))
    print(sum(L_parc_evol_car[-1]))
    print([x/sum(L_parc_evol_bus[-1]) for x in L_parc_evol_bus[-1]])
    print([x/sum(L_parc_evol_car[-1]) for x in L_parc_evol_car[-1]])
filename="Motorisations_bus.csv"
with open(filename,"w",newline='',encoding="utf-8") as f:
    fwrite=csv.writer(f, delimiter=';',quotechar='|')
    fwrite.writerow(["Année"]+L_motorisation)
    for year in range(2021,2061):
        fwrite.writerow([year]+L_parc_evol_bus[year-2021])
filename="Motorisations_car.csv"
with open(filename,"w",newline='',encoding="utf-8") as f:
    fwrite=csv.writer(f, delimiter=';',quotechar='|')
    fwrite.writerow(["Année"]+L_motorisation)
    for year in range(2021,2061):
        fwrite.writerow([year]+L_parc_evol_car[year-2021])

2021
27461
66043
[0.031644878190888896, 0.00276756126870835, 0.0, 0.12727140308073268, 0.83831615745967]
[0.0010144905591811396, 0.0, 0.0, 0.00884272368002665, 0.9901427857607922]
2022
27712
66181
[0.05221564665127021, 0.002453810623556582, 0.0002886836027713626, 0.1574769053117783, 0.7875649538106235]
[0.0010728154606306946, 0.0, 0.0, 0.01905380698387755, 0.9798733775554918]
2023
27976
66355
[0.07488561624249357, 0.0021804403774664, 0.0011080926508435802, 0.19002001715756361, 0.7318058335716329]
[0.0016426795267877325, 0.0, 0.00037676135935498455, 0.033245422349483834, 0.9647351367643734]
2024
28257
66541
[0.09947977492302792, 0.0019464203560179778, 0.0025480411933326255, 0.22550164560993735, 0.6705241179176841]
[0.0028704107242151456, 0.0, 0.0011722096151245097, 0.05174253467786778, 0.9442148449827925]
2025
28540
66719
[0.12529782761037142, 0.0016117729502452699, 0.004519971969166083, 0.2623686054660126, 0.6062018220042046]
[0.004691317315907013, 0.0, 0.0023981174777799427, 0.0747763

In [12]:
# Execution PL
L_base_pli=deepcopy(L_base_pl)
print(2021)
L_parc_evol_pl=[calc_parc(L_base_pli)]
print(sum(L_parc_evol_pl[-1]))
print([x/sum(L_parc_evol_pl[-1]) for x in L_parc_evol_pl[-1]])
for year in range(2022,2061):
    L_base_pli=evol(L_base_pli,L_pl_new,L_pl_years_new,N_new_pl_2021,N_new_pl,duree_vie_pl,k_pl,taux_grow_pl,year)
    print(year)
    L_parc_evol_pl.append(calc_parc(L_base_pli))
    print(sum(L_parc_evol_pl[-1]))
    print([x/sum(L_parc_evol_pl[-1]) for x in L_parc_evol_pl[-1]])
filename="Motorisations_pl.csv"
with open(filename,"w",newline='',encoding="utf-8") as f:
    fwrite=csv.writer(f, delimiter=';',quotechar='|')
    fwrite.writerow(["Année"]+L_motorisation)
    for year in range(2021,2061):
        fwrite.writerow([year]+L_parc_evol_pl[year-2021])

2021
600280
[0.00022989271673219165, 0.0, 0.0, 0.010999866728859865, 0.988770240554408]
2022
602102
[0.00033383048054980716, 0.0, 6.643392647757357e-05, 0.013417992300307921, 0.9861817432926647]
2023
604208
[0.001451486905171729, 0.0, 0.000731536159733072, 0.020633622858353413, 0.9771833540767418]
2024
606380
[0.003567070154028827, 0.0, 0.001990501005969854, 0.03253405455325044, 0.9619083742867509]
2025
608563
[0.006656664963200195, 0.0, 0.0038286915241314374, 0.049053261535781836, 0.9404613819768866]
2026
610754
[0.010645857415587946, 0.0, 0.006211993699590997, 0.0698628252946358, 0.9132793235901853]
2027
612717
[0.015754418434611738, 0.0, 0.00924407189616087, 0.0902129368044301, 0.8847885728647973]
2028
614922
[0.021906843469578257, 0.0, 0.01286992496609326, 0.10977977694732015, 0.8554434546170083]
2029
617098
[0.02907479849229782, 0.0, 0.017141523712603184, 0.1286489342049399, 0.8251347435901591]
2030
619320
[0.037177872505328426, 0.0, 0.02189175224439708, 0.14665762449137765, 0.794